In [1]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import sklearn  as skl
from numpy import asarray

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline      
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

set_config(display='diagram') # Useful for display the pipeline
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier
from sklearn.svm           import SVC

In [2]:
data = pd.read_csv(r'C:\Users\rnr31\Documents\GitHub\apple_watch_tracker\Datasets\dataset_0.5sec.csv')

In [3]:
oe = OrdinalEncoder()
ct = asarray(data['user'])
data['user'] = oe.fit_transform(ct.reshape(-1,1))
data = data.sort_values(by='user')
data

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
62584,62584,84508,630.0,NaN,100.0,15.892297,15.892297,15.892297,3.606631,0.838142,...,89.730876,89.730876,89.730876,NaN,0.75,0.75,0.75,NaN,Walking,0.0
21106,21106,8609,234.0,NaN,100.0,10.142241,10.142241,10.142241,0.007413,0.840593,...,89.770732,89.770732,89.770732,NaN,13.50,13.50,13.50,NaN,Car,0.0
21105,21105,22422,1727.0,NaN,8.0,10.033811,9.701954,10.334343,0.317356,0.890862,...,89.794108,89.794108,89.794108,NaN,20.50,20.50,20.50,NaN,Car,0.0
21104,21104,11240,1151.0,NaN,100.0,10.044192,9.843447,10.144604,0.173851,0.772161,...,NaN,NaN,NaN,NaN,32.50,32.50,32.50,NaN,Car,0.0
21103,21103,7828,289.0,NaN,85.0,8.514446,8.514446,8.514446,0.062750,0.734319,...,89.738523,89.738523,89.738523,NaN,12.25,12.25,12.25,NaN,Car,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37690,37690,129745,309.0,NaN,8.0,10.805771,10.805771,10.805771,0.102640,NaN,...,71.595672,71.595672,71.595672,NaN,NaN,NaN,NaN,NaN,Train,12.0
37689,37689,129760,324.0,NaN,8.0,10.389200,10.389200,10.389200,0.019366,NaN,...,68.089832,68.089832,68.089832,NaN,NaN,NaN,NaN,NaN,Train,12.0
37688,37688,131014,1578.0,NaN,8.0,10.718977,10.704910,10.733045,0.019895,NaN,...,70.312179,70.312179,70.312179,NaN,NaN,NaN,NaN,NaN,Train,12.0
37699,37699,130782,1346.0,NaN,8.0,10.551876,10.463580,10.686567,0.118512,NaN,...,68.048667,68.048667,68.048667,NaN,NaN,NaN,NaN,NaN,Train,12.0


In [4]:
def col_names_change(data):
    data.drop(['Unnamed: 0','id', 'activityrecognition#0'],axis=1, inplace=True)
    data_col = ['time','activityrecognition_1']
    for i in data.columns[2:58]:
        b = i.split('.')[2].split('#')
        data_col.append(f'{b[0]}_{b[1]}')

    for i in data.columns[58:66]:
        b = i.split('#')
        data_col.append(f'{b[0]}_{b[1]}')

    data_col.append('target')
    data_col.append('user')
    data = pd.DataFrame.from_records(data.values)
    data.columns = data_col

    return data

In [5]:
data = col_names_change(data)

In [6]:
data.drop(['pressure_mean', 'pressure_max', 'pressure_min', 'pressure_std', 'sound_std', 'speed_std','step_counter_mean','step_counter_std','light_std',
            'proximity_mean','proximity_min','proximity_max','proximity_std', 'step_counter_min', 'step_counter_max' ], axis=1, inplace=True)


In [7]:
data['target']= data['target'].apply({'Bus':0, 'Car':1, 'Still':2, 'Train':3, 'Walking':4}.get)

In [8]:
df0 = data[data['target']==0]
df1 = data[data['target']==1]
df2 = data[data['target']==2]
df3 = data[data['target']==3]
df4 = data[data['target']==4]
#msno.matrix(df1)

In [9]:
def fill_nan_values(tsdf):
    for i in tsdf.columns:
        ndf = tsdf.groupby(['target'])[f'{i}'].mean()
        mn = ndf.values[0]
        tsdf[f'{i}'] = tsdf[f'{i}'].fillna(mn)
    return tsdf

df0 = fill_nan_values(df0)
df1 = fill_nan_values(df1)
df2 = fill_nan_values(df2)
df3 = fill_nan_values(df3)
df4 = fill_nan_values(df4)



C:\Users\rnr31\AppData\Local\Temp\ipykernel_17512\154742137.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tsdf[f'{i}'] = tsdf[f'{i}'].fillna(mn)


In [10]:
data = pd.concat([df0, df1, df2, df3, df4])
data.shape

(62585, 53)

In [11]:
data

,time,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,...,rotation_vector_max,rotation_vector_std,sound_mean,sound_min,sound_max,speed_mean,speed_min,speed_max,target,user
13713,821.0,100.0,9.580663,9.580663,9.580663,0.012232,0.801618,0.801618,0.801618,0.000162,...,0.928278,0.000081,85.121360,85.121360,85.121360,8.000000,8.000000,8.000000,0,0.0
13714,13.0,100.0,10.958598,9.868953,12.048243,1.540991,0.779063,0.779063,0.779063,0.004365,...,0.482034,0.008963,83.843193,83.843144,83.843242,5.341548,5.341548,5.341548,0,0.0
13715,12.0,100.0,10.958598,9.868953,12.048243,1.540991,0.779063,0.779063,0.779063,0.004365,...,0.452277,0.004023,83.843193,83.843144,83.843242,5.341548,5.341548,5.341548,0,0.0
13716,11.0,100.0,8.841528,6.835641,10.847415,2.836753,0.779063,0.779063,0.779063,0.004365,...,0.452277,0.004023,83.843193,83.843144,83.843242,5.341548,5.341548,5.341548,0,0.0
13717,10.0,100.0,11.618934,9.715548,13.385976,1.839009,0.779063,0.779063,0.779063,0.004365,...,0.417588,0.004023,83.843193,83.843144,83.843242,5.341548,5.341548,5.341548,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62145,483.0,100.0,10.265969,8.937641,11.594297,1.878540,0.870339,0.862206,0.876766,0.015529,...,0.972661,0.016987,73.757710,73.757710,73.757710,1.178197,1.178197,1.178197,4,12.0
62146,418.0,100.0,14.321415,14.321415,14.321415,3.729311,0.870339,0.862206,0.876766,0.015529,...,0.879428,0.004941,79.568839,79.567896,79.569832,1.178197,1.178197,1.178197,4,12.0
62147,546.0,100.0,25.482379,25.482379,25.482379,1.884572,0.870339,0.862206,0.876766,0.015529,...,0.854721,0.029765,79.568839,79.567896,79.569832,1.178197,1.178197,1.178197,4,12.0
62529,854.0,85.0,13.730206,13.730206,13.730206,1.836736,0.870339,0.862206,0.876766,0.015529,...,0.996823,0.001330,78.116997,78.116997,78.116997,1.178197,1.178197,1.178197,4,12.0


In [12]:
data = data.sort_values(by='user')
data.drop(['user'], axis=1, inplace=True)

In [13]:
data_test = data.iloc[:26000, :]
data_train = data.iloc[26000: , :]

#### Training sets
x_train = data_train.drop(['target'], axis=1)
y_train = data_train['target']

#### Testing sets
x_test = data_test.drop(['target'], axis=1)
y_test = data_test['target']

In [14]:
y_train.unique()

array([2, 4, 1, 0, 3], dtype=int64)

In [15]:
y_test.unique()

array([0, 3, 1, 2, 4], dtype=int64)

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [17]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees": ExtraTreesClassifier(n_estimators=100),
  "Random Forest": RandomForestClassifier(n_estimators=100),
  "AdaBoost": AdaBoostClassifier(n_estimators=100),
  "Skl GBM": GradientBoostingClassifier(n_estimators=100),
  "Skl HistGBM": HistGradientBoostingClassifier(max_iter=100),
  "XGBoost": XGBClassifier(n_estimators=100),
  "LightGBM": LGBMClassifier(n_estimators=100),
  "CatBoost":  CatBoostClassifier(n_estimators=100),
}

In [18]:
tree_classifiers = {name: make_pipeline(model) for name, model in tree_classifiers.items()}

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

In [20]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

results_kfold = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():
    start_time = time.time()
        
    # TRAIN AND GET PREDICTIONS USING cross_val_predict() and x,y
    pred = cross_val_predict(model, x_train,y_train, cv=skf)

    total_time = time.time() - start_time

    results_kfold = results_kfold.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_train, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_train, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
results_ord_kfold = results_kfold.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord_kfold.index += 1 
results_ord_kfold.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

C:\Users\rnr31\anaconda3\envs\ds22\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:00:09] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\rnr31\anaconda3\envs\ds22\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:00:30] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\rnr31\anaconda3\envs\ds22\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Users\rnr31\anaconda3\envs\ds22\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:02:31] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\rnr31\anaconda3\envs\ds22\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:02:49] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.5
0:	learn: 0.5905413	total: 532ms	remaining: 52.6s
1:	learn: 0.4085232	total: 643ms	remaining: 31.5s
2:	learn: 0.2940622	total: 758ms	remaining: 24.5s
3:	learn: 0.2207702	total: 873ms	remaining: 20.9s
4:	learn: 0.1724535	total: 993ms	remaining: 18.9s
5:	learn: 0.1416377	total: 1.13s	remaining: 17.7s
6:	learn: 0.1179129	total: 1.28s	remaining: 17s
7:	learn: 0.0972149	total: 1.41s	remaining: 16.2s
8:	learn: 0.0828114	total: 1.58s	remaining: 15.9s
9:	learn: 0.0711678	total: 1.71s	remaining: 15.4s
10:	learn: 0.0625862	total: 1.85s	remaining: 15s
11:	learn: 0.0529314	total: 2s	remaining: 14.7s
12:	learn: 0.0451364	total: 2.12s	remaining: 14.2s
13:	learn: 0.0424951	total: 2.22s	remai

,Model,Accuracy,Bal Acc.,Time
1,XGBoost,99.991800,99.991683,177.517772
2,Extra Trees,99.986333,99.986301,26.769914
3,LightGBM,99.980866,99.980175,41.564900
4,Skl HistGBM,99.978133,99.977129,121.375839
5,Random Forest,99.975400,99.972708,103.648280
6,Skl GBM,99.975400,99.976121,1904.691019
7,CatBoost,99.964466,99.965908,109.725964
8,Decision Tree,99.614596,99.628005,12.869988
9,AdaBoost,81.363947,79.372995,159.510668


In [21]:
rang = abs(y_train.max()) + abs(y_train.min())
for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    pred = model.predict(x_test)
    total_time = time.time() - start_time
        
    
    
    results = results.append({"Model":    model_name,
                              "Accuracy": accuracy_score(y_test, pred)*100,
                              "Bal Acc.": balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)
                              
                              
results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.